# [Mini Project 2: Local Feature Matching](https://www.cc.gatech.edu/~hays/compvision/proj2/proj2.pdf)

Esta notebook de iPython:  
(1) Carga y redimensiona imágenes  
(2) Encuentra puntos de interés en dichas imágenes                 (usted implementará esto)  
(3) Describe cada punto de interés con un local feature            (usted implementará esto)  
(4) Encuentra matching de features                                 (usted implementará esto)  
(5) Visualiza dichos matches  
(6) Evalua los matches basado en correpondencias de datos validados.

## Set up

In [ ]:
%matplotlib inline
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np

from proj2_code.utils import load_image, PIL_resize, rgb2gray, normalize_img, verify
from IPython.core.debugger import set_trace

# Notre Dame
image1 = load_image('../data/1a_notredame.jpg')
image2 = load_image('../data/1b_notredame.jpg')
eval_file = '../ground_truth/notredame.pkl'

# # Mount Rushmore -- this pair is relatively easy (still harder than Notre Dame, though)
# image1 = load_image('../data/2a_rushmore.jpg')
# image2 = load_image('../data/2b_rushmore.jpg')
# eval_file = '../ground_truth/rushmore.pkl'

# # Episcopal Gaudi -- This pair is relatively difficult
# image1 = load_image('../data/3a_gaudi.jpg')
# image2 = load_image('../data/3b_gaudi.jpg')
# eval_file = '../ground_truth/gaudi.pkl'

scale_factor = 0.5
image1 = PIL_resize(image1, (int(image1.shape[1]*scale_factor), int(image1.shape[0]*scale_factor)))
image2 = PIL_resize(image2, (int(image2.shape[1]*scale_factor), int(image2.shape[0]*scale_factor)))

image1_bw = rgb2gray(image1)
image2_bw = rgb2gray(image2)

plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.imshow(image1)
plt.subplot(1,2,2)
plt.imshow(image2)


# Parte 1: Harris Corner Detector 
## Encuentra puntos distintivos en cada imagen (Szeliski 4.1.1)

El detector de esquinas de Harris y SIFT dependen en gran medida de la información de gradiente de la imagen. Van a implementar `compute_image_gradients()` y luego visualizaremos la magnitud de las gracientes de las imágenes.  

¿Qué áreas tienen las magnitudes mas altas y por qué?

In [ ]:
from proj2_code.part1_harris_corner import compute_image_gradients
from proj2_unit_tests.test_part1_harris_corner import test_compute_image_gradients

print('compute_image_gradients(): ', verify(test_compute_image_gradients))

plt.figure(figsize=(10,5))
plt.axis('off')

Ix, Iy = compute_image_gradients(image1_bw)
gradient_magnitudes = np.sqrt(Ix**2 + Iy**2)
gradient_magnitudes = normalize_img(gradient_magnitudes)
plt.subplot(1,2,1)
plt.title(r'$\sqrt{I_x^2 + I_y^2}$')
plt.imshow( (gradient_magnitudes*255).astype(np.uint8))

Ix, Iy = compute_image_gradients(image2_bw)
gradient_magnitudes = np.sqrt(Ix**2 + Iy**2)
gradient_magnitudes = normalize_img(gradient_magnitudes)
plt.subplot(1,2,2)
plt.title(r'$\sqrt{I_x^2 + I_y^2}$')
plt.imshow( (gradient_magnitudes*255).astype(np.uint8))

Ahora calcularemos los elementos de la matriz M de segundo momento $s_x^2, s_y^2, s_x s_y$ en cada pixel, lo cual agrega información de las gradientes en vecindarios locales. Vamos a usar un filtro Gaussiano 2D para agregar información.

In [ ]:
from proj2_code.part1_harris_corner import second_moments

from proj2_unit_tests.test_part1_harris_corner import (
    test_get_gaussian_kernel_2D_pytorch_peak,
    test_get_gaussian_kernel_2D_pytorch_sumsto1,
    test_get_gaussian_kernel_2D_pytorch,
    test_second_moments
)

print('get_gaussian_kernel_2D_pytorch_peak():', verify(test_get_gaussian_kernel_2D_pytorch_peak))
print('get_gaussian_kernel_2D_pytorch_sumsto1():', verify(test_get_gaussian_kernel_2D_pytorch_sumsto1))
print('get_gaussian_kernel_2D_pytorch():', verify(test_get_gaussian_kernel_2D_pytorch))
print('second_moments():', verify(test_second_moments))

sx2, sy2, sxsy = second_moments(image1_bw, ksize = 7, sigma = 10)

Si comparamos $s_x^2$, $s_y^2$, y $s_x s_y$ con $I_x$ y $I_y$, tenemos:

In [ ]:
from proj2_code.utils import normalize_img

plt.figure(figsize=(12,9))
Ix, Iy = compute_image_gradients(image1_bw)
plt.subplot(2,3,1); plt.title(r'$I_x$')
plt.imshow( (normalize_img(np.abs(Ix))*255).astype(np.uint8))
plt.subplot(2,3,2); plt.title(r'$I_y$')
plt.imshow( (normalize_img(np.abs(Iy))*255).astype(np.uint8))

plt.subplot(2,3,4)
plt.title(r'$s_x^2$')
plt.imshow( (normalize_img(np.abs(sx2))*255).astype(np.uint8))

plt.subplot(2,3,5)
plt.title(r'$s_y^2$')
plt.imshow( (normalize_img(np.abs(sy2))*255).astype(np.uint8))

plt.subplot(2,3,6)
plt.title(r'$s_xs_y$')
plt.imshow( (normalize_img(np.abs(sxsy))*255).astype(np.uint8))

Nótese que $s_xs_y$ es alto donde tenemos las gradientes fuertes en ambas direcciones x e y (esquinas por ejemplo y la rosa central).

Ahora usaremos estos 'segundos momentos' para calcular el 'Score de Esquinas' R -- un mapa de respuesta a esquinas -- como una función de las imágenes.

In [ ]:
from proj2_code.part1_harris_corner import compute_harris_response_map
from proj2_unit_tests.test_part1_harris_corner import test_compute_harris_response_map

print('compute_harris_response_map(): ', verify(test_compute_harris_response_map))

R = compute_harris_response_map(image1_bw)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(image1_bw, cmap='gray')
plt.subplot(1,2,2)
plt.title(r'$R$')
plt.imshow(R)

Las áreas brillantes son áreas que tienen mas alta posibilidad de ser esquinas.

AHora implementaremos non-max suppression para encontrar máximos locales en el mapa de respuestas o R de 2D.  Una manera simple de hacer non-maximum suppression es simplemente eligiendo un máximo local sobre una ventana de tamaño (u,v). Esto puede lograrse utilizando max-pooling.

In [ ]:
from proj2_code.part1_harris_corner import maxpool_numpy
from proj2_unit_tests.test_part1_harris_corner import test_maxpool_numpy, test_nms_maxpool_pytorch
from proj2_code.utils import verify

print('maxpool_numpy(): ', verify(test_maxpool_numpy))

toy_response_map = np.array(
[
    [1,2,2,1,2],
    [1,6,2,1,1],
    [2,2,1,1,1],
    [1,1,1,7,1],
    [1,1,1,1,1]
]).astype(np.float32)
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plt.imshow(toy_response_map.astype(np.uint8))

plt.subplot(1,2,2)
maxpooled_image = maxpool_numpy(toy_response_map, ksize=3)
plt.imshow(maxpooled_image.astype(np.uint8))

Dada una grilla de 5x5 que contiene scores de respuestas R, non-max suppression nos va a permitir elegir valores que son máximos locales. Si utilizamos como ejemplo la grilla de arriba y solicitamos los $k=2$ puntajes de respuesta obtendriamos (1,1) y (3,3)

In [ ]:
from proj2_code.part1_harris_corner import nms_maxpool_pytorch

print('nms_maxpool_pytorch(): ', verify(test_nms_maxpool_pytorch))

x_coords, y_coords, confidences = nms_maxpool_pytorch(toy_response_map, k=2, ksize=3)
print('Coordinates of local maxima:')
for x, y, c in zip(x_coords, y_coords, confidences):
    print(f'\tAt {x},{y}, local maximum w/ confidence={c:.2f}')

Acá llamaos a la función `get_harris_interest_points()` que está en `part1_harris_corner.py` para detectar puntos 'intersantes' en las imágenes.  

**IMPORTANTE**  
Asegúrate de agregar tu código a la función `get_harris_interest_points()` para obtener los puntos de interés.

In [ ]:
from proj2_unit_tests.test_part1_harris_corner import test_get_harris_interest_points, test_remove_border_vals

print('test_remove_border_vals(): ', verify(test_remove_border_vals))

print('get_harris_interest_points()', verify(test_get_harris_interest_points))

In [ ]:
import copy
from proj2_code.part1_harris_corner import get_harris_interest_points
from proj2_code.utils import show_interest_points

num_interest_points = 2500
X1, Y1, _ = get_harris_interest_points( copy.deepcopy(image1_bw), num_interest_points)
X2, Y2, _ = get_harris_interest_points( copy.deepcopy(image2_bw), num_interest_points)

num_pts_to_visualize = 300
# Visualize the interest points
rendered_img1 = show_interest_points(image1, X1[:num_pts_to_visualize], Y1[:num_pts_to_visualize])
rendered_img2 = show_interest_points(image2, X2[:num_pts_to_visualize], Y2[:num_pts_to_visualize])
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.imshow(rendered_img1)
plt.subplot(1,2,2); plt.imshow(rendered_img2)
print(f'{len(X1)} corners in image 1, {len(X2)} corners in image 2')

# Parte 2: Normalized Patch Feature Descriptor
## Crear feature de vectores para cada punto de interés (Szeliski 7.1.2)

Quizás la manera mas simple de describir un punto de interés es utilizar un parche de 16x16  sobre el punto de interés, convertirlo a un vector de 256 Dimensiones (con los valores de intensidad de la imagen) y luego normalizarlo.

In [ ]:
from proj2_code.part2_patch_descriptor import compute_normalized_patch_descriptors
from proj2_unit_tests.test_part2_patch_descriptor import test_compute_normalized_patch_descriptors

print('compute_normalized_patch_descriptors:', verify(test_compute_normalized_patch_descriptors))

image1_features = compute_normalized_patch_descriptors(image1_bw, X1, Y1, feature_width=16)
image2_features = compute_normalized_patch_descriptors(image2_bw, X2, Y2, feature_width=16)

# Visualizamos como se ven los primeros 300 vectores de features para la imagen 1. (No deben ser todos iguales ni tampoco 
# imágenes negras.)
plt.figure()
plt.subplot(1,2,1); plt.imshow(image1_features[:300])

## Parte 4: Match features (Szeliski 7.1.3, page 423)

In [ ]:
# Testea tu implementacioń de feature matching
from proj2_unit_tests.test_part3_feature_matching import (
    test_match_features_ratio_test,
    test_compute_feature_distances_2d,
    test_compute_feature_distances_10d
)
print('compute_feature_distances (2d):', verify(test_compute_feature_distances_2d))
print('compute_feature_distances (10d):', verify(test_compute_feature_distances_10d))
print('match_features_ratio_test:', verify(test_match_features_ratio_test))

In [ ]:
from proj2_code.part3_feature_matching import match_features_ratio_test

matches, confidences = match_features_ratio_test(image1_features, image2_features)
print('{:d} matches from {:d} corners'.format(len(matches), len(X1)))

## Visualization

Es posible que desee establecer 'num_pts_to_visualize' y 'num_pts_to_evaluate' a una constante (por ejemplo, 100) una vez que comience a detectar cientos de puntos de interés, de lo contrario, los puntos de interés se van a ver todos encimados y desordenadas. También puede establecer un umbral basado en la confianza.

A continuacón, tenemos 2 funciones de visualización. Pueden comentar si quieren uno de ellos para ver mejor el resultado.

In [ ]:
import os
from proj2_code.utils import show_correspondence_circles, show_correspondence_lines
os.makedirs('../results', exist_ok=True)
# num_pts_to_visualize = len(matches)
num_pts_to_visualize = 200
c1 = show_correspondence_circles(image1, image2,
                    X1[matches[:num_pts_to_visualize, 0]], Y1[matches[:num_pts_to_visualize, 0]],
                    X2[matches[:num_pts_to_visualize, 1]], Y2[matches[:num_pts_to_visualize, 1]])
plt.figure(figsize=(10,5)); plt.imshow(c1)
plt.savefig('../results/vis_circles.jpg', dpi=1000)
c2 = show_correspondence_lines(image1, image2,
                    X1[matches[:num_pts_to_visualize, 0]], Y1[matches[:num_pts_to_visualize, 0]],
                    X2[matches[:num_pts_to_visualize, 1]], Y2[matches[:num_pts_to_visualize, 1]])
plt.figure(figsize=(10,5)); plt.imshow(c2)
plt.savefig('../results/vis_lines.jpg', dpi=1000)

Comente la función a continuación si no está probando en los pares de imágenes de Notre Dame, Episcopal Gaudi y Mount Rushmore; esta función de evaluación solo funcionará para aquellas que tengan información de validación disponible.

In [ ]:
from proj2_code.utils import evaluate_correspondence
# num_pts_to_evaluate = len(matches)
num_pts_to_evaluate = 2500
_, c = evaluate_correspondence(image1, image2, eval_file, scale_factor,
                        X1[matches[:num_pts_to_evaluate, 0]], Y1[matches[:num_pts_to_evaluate, 0]],
                        X2[matches[:num_pts_to_evaluate, 1]], Y2[matches[:num_pts_to_evaluate, 1]])
plt.figure(figsize=(8,4)); plt.imshow(c)
plt.savefig('../results/eval.jpg', dpi=1000)

# Parte 4: Sift Feature Descriptor (Szeliski 7.1.2, page 414)
SIFT se basa en calcular las magnitudes y orientaciones de las gradientes de la imagen y luego calcular los histogramas ponderados.

In [ ]:
from proj2_unit_tests.test_part4_sift_descriptor import (
    test_get_magnitudes_and_orientations,
    test_get_gradient_histogram_vec_from_patch
)
print('get_magnitudes_and_orientations:', verify(test_get_magnitudes_and_orientations))

In [ ]:
print('get_gradient_histogram_vec_from_patch():', verify(test_get_gradient_histogram_vec_from_patch))

In [ ]:
from proj2_unit_tests.test_part4_sift_descriptor import test_get_feat_vec, test_get_SIFT_descriptors
print(verify(test_get_feat_vec))
print(verify(test_get_SIFT_descriptors))

In [ ]:
from proj2_code.part4_sift_descriptor import get_SIFT_descriptors
from proj2_code.utils import cheat_interest_points

import time
start = time.time()
image1_features = get_SIFT_descriptors(image1_bw, X1, Y1)
image2_features = get_SIFT_descriptors(image2_bw, X2, Y2)
end = time.time()
duration = end - start
print(f'SIFT took {duration} sec.')

# visualizar cómo se ven los valores de los primeros 200 vectores de features SIFT (no deben ser idénticos o completamente negros)
plt.figure(); plt.subplot(1,2,1); plt.imshow(image1_features[:200])

In [ ]:
from proj2_code.utils import show_correspondence_circles, show_correspondence_lines

matches, confidences = match_features_ratio_test(image1_features, image2_features)
print('{:d} matches from {:d} corners'.format(len(matches), len(X1)))

# num_pts_to_visualize = len(matches)
num_pts_to_visualize = 200
c1 = show_correspondence_circles(
    image1,
    image2,
    X1[matches[:num_pts_to_visualize, 0]],
    Y1[matches[:num_pts_to_visualize, 0]],
    X2[matches[:num_pts_to_visualize, 1]],
    Y2[matches[:num_pts_to_visualize, 1]]
)
plt.figure(figsize=(10,5)); plt.imshow(c1)
plt.savefig('../results/vis_circles.jpg', dpi=1000)
c2 = show_correspondence_lines(
    image1,
    image2,
    X1[matches[:num_pts_to_visualize, 0]],
    Y1[matches[:num_pts_to_visualize, 0]],
    X2[matches[:num_pts_to_visualize, 1]],
    Y2[matches[:num_pts_to_visualize, 1]]
)
plt.figure(figsize=(10,5)); plt.imshow(c2)
plt.savefig('../results/vis_lines.jpg', dpi=1000)

In [ ]:
from proj2_code.utils import evaluate_correspondence
num_pts_to_evaluate = len(matches)
_, c = evaluate_correspondence(
    image1,
    image2,
    eval_file,
    scale_factor,
    X1[matches[:num_pts_to_evaluate, 0]],
    Y1[matches[:num_pts_to_evaluate, 0]],
    X2[matches[:num_pts_to_evaluate, 1]],
    Y2[matches[:num_pts_to_evaluate, 1]]
)
plt.figure(figsize=(8,4)); plt.imshow(c)
plt.savefig('../results/eval.jpg', dpi=1000)

Asegúrese de que su código se ejecuta en menos de 90 segundos y alcanza >80% de accuray en el par de Notre Dame:

In [ ]:
from proj2_unit_tests.test_part4_sift_descriptor import (
    test_feature_matching_speed,
    test_feature_matching_accuracy
)
print('SIFT pipeline speed test:', verify(test_feature_matching_speed))
print('SIFT pipeline accuracy test:', verify(test_feature_matching_accuracy))

# Opcional Puntaje Extra
Implemente SIFT de una manera completamente vectorizada con al menos un 80% de precisión en Notre Dame (sin bucles sobre píxeles, como máximo un bucle sobre puntos clave). SIFT en ambas imágenes debería ejecutarse en menos de 5 segundos. La orientación en cada vector se puede obtener mediante convolución 1x1 con 8 vectores bases o de vector unitario.

In [ ]:
import time
from proj2_code.part4_sift_descriptor import get_sift_features_vectorized

start = time.time()
image1_features = get_sift_features_vectorized(image1_bw, X1, Y1)
image2_features = get_sift_features_vectorized(image2_bw, X2, Y2)
end = time.time()
duration = end - start
print(f'SIFT took {duration} sec.')

matches, confidences = match_features_ratio_test(image1_features, image2_features)

num_pts_to_evaluate = len(matches) - 1
_, c = evaluate_correspondence(
    image1,
    image2,
    eval_file,
    scale_factor,
    X1[matches[:, 0]],
    Y1[matches[:, 0]],
    X2[matches[:, 1]],
    Y2[matches[:, 1]]
)
plt.figure(figsize=(8,4)); plt.imshow(c)
plt.savefig('../results/eval.jpg', dpi=1000)

In [ ]:
from proj2_unit_tests.test_part4_sift_descriptor import test_extra_credit_vectorized_sift

print(verify(test_extra_credit_vectorized_sift))